## Analyzing outputs of 100 parameter sets ensemble simulaitons

Analyzing the first batch of ensemble simulations. <br/>
Simulation outputs stored: /home/disk/eos8/ach315/upscale/sims/opt

In [1]:
import numpy as np
import pandas as pd
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import xarray as xr
import datetime
import seaborn as sns

from scipy.stats.stats import pearsonr
from palettable.colorbrewer.sequential import OrRd_6
from palettable.colorbrewer.sequential import YlGn_9
from palettable.colorbrewer.sequential import YlGnBu_8
from palettable.colorbrewer.sequential import RdPu_5

In [3]:
params = pd.read_csv('/home/disk/eos8/ach315/upscale/inits/param_100.csv')
params.head()

,staygreen,rmax_lir,rmax_ltar,juv_leaves,leaf_wid,WLRATIO,A_LW,LAF,f,scatt,vcm_25,vpm_25,g0,g1,t_opt,psi_threshold
0,4.192327,0.170936,1.116221,19,0.060117,1.578166,0.578437,1.082704,0.374039,0.799075,41.453395,63.020343,0.050014,4.262101,23.156285,-3.209627
1,5.085099,0.458642,0.532489,13,0.053712,1.593230,0.944650,2.500608,0.084047,0.638286,55.254491,76.360963,0.045041,9.476365,21.452484,-4.528151
2,5.462802,0.409265,1.126607,19,0.055231,1.452949,0.615309,1.305084,0.209307,0.547004,40.314858,68.638982,0.071773,3.290494,25.081642,-1.387541
3,2.468920,0.159873,0.209975,14,0.021548,1.951003,0.603160,1.430029,0.198690,0.216546,53.057031,71.797256,0.102321,3.169339,26.865773,-3.101738
4,2.217561,0.478633,0.463970,16,0.045654,1.027741,0.790700,1.147568,0.793187,0.582211,50.710891,78.179130,0.099021,6.539257,27.086055,-1.328671


In [2]:
# manual update of maizsim output column name
cols = ['jday', 'time', 'leaves', 'mature_lvs', 'drop_lvs', 'LA', 'LA_dead', 'LAI', 'RH', 
        'leaf_WP', 'PFD', 'solrad', 'temp_soil', 'temp_air', 'temp_can', 'ET_dmd', 'ET_suply',
        'Pn', 'Pg', 'resp', 'av_gs', 'LAI_sunlit', 'LAI_shaded', 'PFD_sunlit', 'PFD_shaded', 
        'An_sunlit', 'An_shaded', 'Ag_sunlit', 'Ag_shaded', 'gs_sunlit', 'gs_shaded', 'VPD',
        'N', 'N_dmd', 'N_upt', 'N_leaf', 'PCRL', 'dm_total', 'dm_shoot', 'dm_ear', 'dm_totleaf', 'dm_dropleaf', 
        'df_stem', 'df_root', 'roil_rt', 'mx_rootdept', 'available_water', 'soluble_c', 'note']

maybe need to think about storing all the data in an xarray format:


In [6]:
years = np.arange(1961,1963)
cultivars = np.arange(0,100)
yrs = []
cvars = []
sites = []
dm_ears = []


for i in years:
    for j in cultivars:
        sims = glob.glob('/home/disk/eos8/ach315/upscale/sims/opt/' + str(i) + '/var_' + str(j) + '/out1*.txt')
        for sim in sims:
            site = sim.split('/')[-1].split('_')[1]
            df = pd.read_csv(sim)
            df.columns = cols
            ear = df.dm_ear.max()
            yrs.append(i)
            cvars.append(j)
            sites.append(site)
            dm_ears.append(ear)
            
        print(i, j)

1961 0
1961 1
1961 2
1961 3
1961 4
1961 5
1961 6
1961 7
1961 8
1961 9
1961 10
1961 11
1961 12
1961 13
1961 14
1961 15
1961 16
1961 17
1961 18
1961 19
1961 20
1961 21
1961 22
1961 23
1961 24
1961 25
1961 26
1961 27
1961 28
1961 29
1961 30
1961 31
1961 32
1961 33
1961 34
1961 35
1961 36
1961 37
1961 38
1961 39
1961 40
1961 41
1961 42
1961 43
1961 44
1961 45
1961 46
1961 47
1961 48
1961 49
1961 50
1961 51
1961 52
1961 53
1961 54
1961 55
1961 56
1961 57
1961 58
1961 59
1961 60
1961 61
1961 62
1961 63
1961 64
1961 65
1961 66
1961 67
1961 68
1961 69
1961 70
1961 71
1961 72
1961 73
1961 74
1961 75
1961 76
1961 77
1961 78
1961 79
1961 80
1961 81
1961 82
1961 83
1961 84
1961 85
1961 86
1961 87
1961 88
1961 89
1961 90
1961 91
1961 92
1961 93
1961 94
1961 95
1961 96
1961 97
1961 98
1961 99
1962 0
1962 1
1962 2
1962 3
1962 4
1962 5
1962 6
1962 7
1962 8
1962 9
1962 10
1962 11
1962 12
1962 13
1962 14
1962 15
1962 16
1962 17
1962 18
1962 19
1962 20
1962 21
1962 22
1962 23
1962 24
1962 25
1962 26
1962